In [1]:
#importing libraries and reading file
import csv
f=open("irisself.txt",'r')
reader = csv.reader(f)
import numpy as np
import pandas as pd
import random
import random
df=pd.read_csv('irisself.txt')


In [2]:
#creating training and testing data
#input parameters: a in dataframe and percent of test data reuired
def testing_training_set(a,percent):
    number=((percent/100)*len(a)) #calculatinting number of testcases required
    number=round(number)     #converting it into an integer#
    
    #getting index of dataframe
    index=a.index.tolist()  
    #getting random number of indexes
    random_index=random.sample(index,number) 
    #removing indexes
    training_data=a.drop(random_index)  
    testing_data=a.loc[random_index]    #getting testing data
    global feat   #creating global feat
    feat=a.columns
    return training_data, testing_data

In [3]:
#creating training data and testing data
training_data,testing_data=testing_training_set(df,30) 

#converting data to numpy array for easy use
data_np=training_data.values
test_np=testing_data.values

In [4]:
# checking if entropy is zero
def is_entropy_zero(a):
        label_list=a[0:-1,-1]  #getting label column
        unique_labels=set(label_list) #creating set, so that duplicate values are removed and we get unique values
        if (len(unique_labels))>=2: #if number of unique labels is greater than 1, then entropy cannot be zero
            return False #boolean outputs to be used directly
        else:
            return True

In [5]:
is_entropy_zero(data_np)

False

In [6]:
#returning the label with maximum count
#incase of a tie, it returns the first value


#getting labels on the classified data
def put_label(a):
    labels=a[:,-1] #getting labels
    unique_labels=list(set(labels)) #making it set for unique values and then makin it a list for easy access
    count=np.zeros((((len(unique_labels))),), dtype=int) #creating empty arrays of zero integer
    count=list(count)   #
    #print(len(unique_labels))
    
#getting maxximum count of a label
    for x in range(0,(len(unique_labels))):
        #print(x)
        for y in range(0,(len(a))):
           if (unique_labels[x-1] == labels[y-1]):
                count[x-1]=count[x-1]+1
    #print(count)
    maximum_count=max(count)
    return_index=(count.index(maximum_count)) #getting index of label with maxm count
    return_label=unique_labels[return_index]  
    return return_label


In [7]:
#wheter a category is continuous or category wise
def continuous_or_categorical_data(b):
    all_c=b.columns #getting all columns
    #print(all_c)
    all_c_with_cont_data = b._get_numeric_data().columns #columns with numeric values
    #print(all_c_with_cont_data)
    
    l=[]
    for x in all_c:
        if x in all_c_with_cont_data:
            l.append("continuous")
        else:
            l.append("category")
    l.pop()
    #print(l)
    return l
    

In [8]:
global feat_category
feat_category=continuous_or_categorical_data(training_data)

In [9]:
#creating possible boundaries for splitting of data
def division_boundaries2(a):
    divide = {}
    c = len(a[0]) #no of columns
    for z in range(1,c): #looping for all columns exxcept last one
        y = a[:, z-1]
        uni = np.unique(y)
        divide[z-1]=uni
    return divide

In [10]:
#division_boundaries2(data_np)

In [11]:
#diving data based on column number and value
def divide_data(a,c,v):
    sv=a[:,c] #getting vvalue of column
    d=feat_category[c] #type of column values #category or #continuous
    if d == "continuous":
        yes = a[sv <= v]
        no = a[sv > v]
    elif d =="category": 
        yes = a[sv == v]
        no = a[sv != v]
    return yes, no
   

In [12]:
#getting entropy
def get_ent(a):
    labels=a[:,-1]
    unique_labels=list(set(labels))
    count=np.zeros((((len(unique_labels))),), dtype=int) #removed -1
    count=list(count)
    #print(len(unique_labels))
    

    for x in range(0,(len(unique_labels))):
        #print(x)
        for y in range(0,(len(a))):
           if (unique_labels[x-1] == labels[y-1]):
                count[x-1]=count[x-1]+1
    #print(count)
    y=sum(count)
    count[:] = [x / y for x in count]
    z=count
    #print(count)
    z1=np.log2(z)
    #print(z1)
    rez=(z*z1)
    rez=-sum(rez)
    #print(rez)
    #rez=rez+z[i]
    return rez
    
    

In [13]:
#getting weighted entropy
#weighted E= p1*E1+p2*E2
def total_entropy(d1,d2):
    l1=len(d1)
    l2=len(d2)
    l=l1+l2
    p1=len(d1)/l
    p2=len(d2)/l
    E1=p1*get_ent(d1)
    E2=p2*get_ent(d2)
    Etotal=(E1+E2)
    return Etotal

In [14]:
#deciding the best division
def optimal_division(d,s): #getting data  and possible divisions
        E=10000  #setting very high values of initial entropy
        for x in s: #
            for y in s[x]:
                d1,d2=divide_data(d,x,y) #dividing data
                newE=total_entropy(d1,d2) #getting weighted entropy
                #getting value and col for least entropy
                if E > newE:  #if new entropy is less
                    E=newE  #making it equal to new entropy
                    
                    val=y  #returning these values
                    col=x
        return col,val
                

In [15]:
#creating final binary tree

def create_binary_T(df):
    
    data = df           
    #if entrop is zero, it is a leaf node
    if (is_entropy_zero(data)) :
        label = put_label(data)  #getting output label for this branch
        return label

    else:    
        #creating new subtree
        division_values = division_boundaries2(data) #possible_values of division
        div_c, div_v = optimal_division(data, division_values) #column and values for best division of data
        div_1, div_2 = divide_data(data, div_c, div_v) #divided data output
        
        #feature on which division is based
        feature_name = feat[div_c]
        
        #creating key-value pair :
        #feature name and feature value for division
        
        node = "feature: {} ,valueupto: {} ,Label:".format(feature_name, div_v)
        #node = "{} <= {}".format(feature_name, div_v)
        #recursion part
        subnode_1 = create_binary_T(div_1) #creating further sub nodes untill data is completely classified
        subnode_2 = create_binary_T(div_2)    #second branch is made, as data was not pure   
        
        #making sub node
        sub_node = {node: []}
        #inserting the new sub nodes at the end of the node
        sub_node[node].insert(len(sub_node[node]),subnode_1) #left subnode
        sub_node[node].insert(len(sub_node[node]),subnode_2)  #right subnode

        
        return sub_node

In [16]:
tree = create_binary_T(data_np)
print(tree)

{'feature: PetalLength ,valueupto: 1.9 ,Label:': ['Iris-setosa', {'feature: PetalWidth ,valueupto: 1.7 ,Label:': [{'feature: PetalLength ,valueupto: 4.9 ,Label:': ['Iris-versicolor', {'feature: PetalWidth ,valueupto: 1.5 ,Label:': ['Iris-virginica', 'Iris-versicolor']}]}, {'feature: PetalLength ,valueupto: 4.8 ,Label:': [{'feature: SepalLength ,valueupto: 5.9 ,Label:': ['Iris-versicolor', 'Iris-virginica']}, 'Iris-virginica']}]}]}


### choosing features randomly(upto 2 features)

In [17]:
def rand_forest_data(a):
    
    t=np.zeros((105,3),dtype=object)
    x=random.sample(range(0,4),3)
    t[:,0]=a[:,x[0]]
    t[:,1]=a[:,x[1]]
    t[:,-1]=a[:,-1]
    return t

In [18]:
random_for_data=rand_forest_data(data_np)
    

### getting output from tree

In [29]:
def applying_tree(input_data,binary_T):
    question = list(binary_T.keys())[0]
    #print(question)
    _,attribute,_,value,_ = question.split(" ")
    print(value,attribute,input_data[attribute])
    if input_data[attribute] <= float(value) or str(input_data[attribute]) == value:
        answer = binary_T[question][0]
    else:
        answer = binary_T[question][1]
    
    if not isinstance(answer, dict):
        return answer
    
    else:
        residual_tree = answer
        return applying_tree(input_data, residual_tree)
    

### calculating accuracy

In [20]:
def accuracy(df,T):
    df["net_result"]=df.apply(applying_tree,args=(T,))
    df["true"] = df["net_result"] == df["label"]
    
    total_acc = df["true"].mean()
    
    return total_acc

In [21]:
example = training_data.iloc[0]
example

SepalLength            5.1
SepalWidth             3.5
PetalLength            1.4
PetalWidth             0.2
label          Iris-setosa
Name: 0, dtype: object

In [30]:
tree = create_binary_T(data_np)
applying_tree(example,tree)

1.9 PetalLength 1.4


'Iris-setosa'

## Applying random forest:
### (choosing random datasets, random features, creating multiple trees,applying test data over all trees, storing     output for all trees for getting majority result)

In [23]:
def rand_forest(x_df):  #add testing data
    x_np=x_df.values
    l=[]
    l2=[]
    _,final_test_df=testing_training_set(x_df,30)
    for i in range(1,5):
        x1_df,x2_df=testing_training_set(x_df,30)
        x1_np=x1_df.values
        x2_np=x2_df.values
        t=rand_forest_data(x1_np)
        tree1=create_binary_T(t)
        print(tree1)
        #x2_df["output"]=x2_df.apply(applying_tree,args=(tree1,),axis=1,raw=True)
        #temp=x2_df.apply(applying_tree,args=(tree1,),axis=1,raw=True)
        temp=final_test_df.apply(applying_tree,args=(tree1,),axis=1,raw=True)
        
        l.extend(temp)
    l2.extend(x2_df.iloc[:,-1])
    #put testing data to tree and get output
    #keep appending output
    #use reshape
    #print(l)
    ##print(l2)
    #t=rand_forest_data(a)
    #tree2=create_binary_T(t)
    #t=rand_forest_data(a)
    #tree3=create_binary_T(t)
    return l,l2

In [24]:
#def rand_forest2(a):  #add testing data
#    
 #   t=rand_forest_data(a)
 #   tree1=create_binary_T(t)
 #   
 #   t=rand_forest_data(a)
 #   tree2=create_binary_T(t)
 ##   t=rand_forest_data(a)
  #  tree3=create_binary_T(t)
  #  return tree1,tree2,tree3

In [25]:
#rand_forest(data_np,test_np)
l1,l2=rand_forest(df)

#l1=l1[:,-1]
#l2=l2[:,-1]
#count=0
#for x in range(0,len(l1)):
 #   if l1[x] == l2[x]:
 #       count=count+1
#accura=count/len(l1)
#print(accura)

{'feature: SepalLength ,valueupto: 0.6 ,Label:': ['Iris-setosa', {'feature: SepalLength ,valueupto: 1.6 ,Label:': [{'feature: SepalWidth ,valueupto: 4.9 ,Label:': ['Iris-versicolor', {'feature: SepalLength ,valueupto: 1.5 ,Label:': ['Iris-virginica', 'Iris-versicolor']}]}, {'feature: SepalWidth ,valueupto: 4.8 ,Label:': [{'feature: SepalLength ,valueupto: 1.7 ,Label:': ['Iris-virginica', 'Iris-virginica']}, 'Iris-virginica']}]}]}
5.2
5.2
3.4
5.2
5.8
5.8
2.6
5.8
6.7
6.7
3.3
6.7
6.5
6.5
2.8
6.5
5.5
5.5
2.4
5.5
5.6
5.6
3.0
5.6
4.8
4.8
3.1
4.8
5.7
5.7
2.5
5.7
5.8
5.8
2.7
5.8
4.8
4.8
3.0
4.8
6.4
6.4
2.8
6.4
5.6
5.6
2.9
5.6
6.3
6.3
3.3
6.3
5.4
5.4
3.0
5.4
6.2
6.2
2.8
6.2
7.3
7.3
2.9
7.3
5.8
5.8
2.8
5.8
7.7
7.7
3.0
7.7
7.1
7.1
3.0
7.1
5.7
5.7
3.8
5.7
4.4
4.4
3.0
4.4
4.6
4.6
3.2
4.6
5.7
5.7
2.9
5.7
6.5
6.5
3.0
6.5
5.0
5.0
3.4
5.0
5.4
5.4
3.9
5.4
5.4
5.4
3.9
5.4
6.7
6.7
3.1
6.7
6.2
6.2
3.4
6.2
7.7
7.7
3.8
7.7
6.0
6.0
2.9
6.0
6.3
6.3
2.5
6.3
5.8
5.8
4.0
5.8
5.5
5.5
4.2
5.5
4.9
4.9
3.0
4.9
5.7
5.

In [26]:
l1_dash=np.reshape(l1,(45,4))
indi=l1_dash.argmax(axis=1)
#print(indi)
#final=[None]*45
#for x in range(1,len(indi)):
#    final[x]=l1_dash[x,indi]
#print(final)

## getting predicted output and actual output
### calculating accuracy for random forest

In [27]:
#print(len(l2))
final_rand_output=[]
final_rand_output2=[]
for x in range(0,180,4):
    temp2=l1[x:x+4]
    i=np.argmax(temp2)
    #print(temp2[i])
    final_rand_output.insert(-1,temp2[i])
#print(l2)
print(final_rand_output)
print(l2)
count=0
for x in range(0,len(final_rand_output)):
    if final_rand_output[x]==l2[x]:
        count=count+1
print(count/len(l2))

['Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica']
['Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa', 'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-virginica', 'I

In [28]:
#commented entire paragraph

#tree = create_binary_T(data_np)
#l1=[]
#l2=training_data.iloc[:,-1]
#testing_data["output"]=testing_data.apply(applying_tree,args=(tree,),axis=1)
#l1=testing_data.apply(applying_tree,args=(tree,),axis=1)
#print(l1)
#print(l2)
#applying_tree(testing_data,tree)
#accu = accuracy(testing_data, tree)
#print(accu)
#accuracy